<img src="https://pycon.hk/wp-content/uploads/2018/10/pyconhk-logo-R-1038x576.jpg" width="25%"></img>

# Geospatial Data Processing using Python

Prof. Martin Christen<br/>
University of Applied Sciences and Arts Northwestern Switzerland<br/>
Institute Geomatics

### Github

This notebook can be downloaded here:
https://github.com/martinchristen/pyconhk-2020

### Installation

Download Anaconda: https://www.anaconda.com/products/individual <br/>
Use the Anaconda Prompt to enter everything below.

1. Create the virtual environment named "geopython" and activate it:

       conda create --name geopython python=3.7 -y
       conda activate geopython


2. Add this environment to Jupyter (for example using "GeoPython" as name)

       conda install ipykernel -y
       python -m ipykernel install --user --name GeoPython
   
   
3. Install geospatial modules (we will add more later)

       conda install gdal rasterio matplotlib geopandas -y
       conda install geoplot folium osmnx folium -c conda-forge -y


#### Jupyter Bug

When creating custom Kernels there is a bug in Jupyter (Bug: https://github.com/jupyter/notebook/issues/4569) 

We have to manually add to the PATH environment, and we also have to set "PROJ_LIB".<br/>
(you could also add this to ~/.ipython/profile_default/startup/ipython_startup.py )<br/>
<br/>
I prepared a function in geoutils.py called "fixenv" to fix the PATH and PROJ_LIB.


In [ ]:
# fix dll/so problem
import geoutils
geoutils.fixenv()

In [ ]:
%matplotlib inline

## What is Geospatial Data ?

Geospatial data is data with a spatial component - it describes objects with a spatial reference to the earth’s surface. This data consists of a spatial component (**where**), attributes (**what**), and often a time reference (**when**).

Geospatial data is multidimensional, we need at least two coordinates to define a position (for example x and y). Geospatial data can be huge - some datasets may be in the terabyte to petabyte range, which also makes it really hard to display it or to update it. 
And when we want to view the data it is usually projected to a flat surface. There are two main methods of representing geospatial data digitally. Both methods try to reduce the reality of something we can store on the computer, the first kind is called **raster data** and the second **vector data**.



## Vector Data

### Introduction to Shapely

http://toblerity.org/shapely
http://toblerity.org/shapely/manual.html

Shapely is a BSD-licensed Python package for **manipulation** and **analysis** of **planar geometric objects**. 

* Shapely is **not** concerned with data formats or coordinate systems.
* Shapely is based on the widely deployed GEOS (the engine of PostGIS) and JTS (from which GEOS is ported) libraries.

There is also PyGEOS https://pypi.org/project/pygeos/ - I'm not using it at this time.

### Simple Feature Access

http://www.opengeospatial.org/standards/sfa

https://en.wikipedia.org/wiki/Simple_Features


**Simple Feature Access** is both an Open Geospatial Consortium (OGC) and International Organization for Standardization (ISO) standard **ISO 19125** that specifies a common storage and access model of mostly two-dimensional geometries (point, line, polygon, multi-point, multi-line, etc.) used by geographic information systems.

Shapely supports the following Features:

* Point
* LineString
* LinearRing          
* Polygon
* MultiLineString
* MultiPoint
* MultiPolygon

![image](img/features.png)

In [ ]:
from shapely.geometry import Polygon, Point, MultiPolygon

polygon1 = Polygon([(30, 10), (40, 40), (20, 35), (10, 20), (30, 10)])

print(f"Polygon area: {polygon1.area}, polygon length: {polygon1.length}") 

In [ ]:
polygon1

In [ ]:
# plotting using cartopy:
import plotshape

In [ ]:
plotshape.plot(polygon1);

In [ ]:
polygon2 = Polygon([(20,20),(80,30),(50,40),(20,20)])
plotshape.plot(polygon2);

In [ ]:
union = polygon2.union(polygon1)
plotshape.plot(union);

In [ ]:
intersecion = polygon2.intersection(polygon1)
plotshape.plot(intersecion);

In [ ]:
result = polygon2.symmetric_difference(polygon1)
plotshape.plot(result);

In [ ]:
s = result.wkt
s

In [ ]:
import shapely.wkt

mypolygon = shapely.wkt.loads(s)
plotshape.plot(mypolygon);

There are also several binary operations available:

- **contains** (Returns True if the interior of the object intersects the interior of the other but does not contain it, and the dimension of the intersection is less than the dimension of the one or the other.)
- **intersects** (Returns True if the boundary and interior of the object intersect in any way with those of the other.)
- **witin** (Returns True if the object’s boundary and interior intersect only with the interior of the other (not its boundary or exterior).
- **touches** (Returns True if the objects have at least one point in common and their interiors do not intersect with any part of the other.)
- **crosses** (Returns True if the interior of the object intersects the interior of the other but does not contain it, and the dimension of the intersection is less than the dimension of the one or the other.)
- **equals** (Returns True if the set-theoretic boundary, interior, and exterior of the object coincide with those of the other.)

In [ ]:
polygon1.intersects(polygon2)

In [ ]:
polygon1.within(polygon2)

### Let's get some data!

We download and unzip the geopackage from https://www.naturalearthdata.com/

In [ ]:
import os

if not os.path.exists("geodata"):
    os.mkdir("geodata")

In [ ]:
import geoutils

if not os.path.exists("geodata/ne.gpkg.zip"):
    ne = geoutils.geodata["natural-earth"] # URL to Natural Earth Dataset (GeoPackage Format)
    geoutils.download(ne, "geodata/ne.gpkg.zip")
else:
    print("dataset is already downloaded")

In [ ]:
import zipfile

if not os.path.exists("geodata/packages/natural_earth_vector.gpkg"):
    with zipfile.ZipFile("geodata/ne.gpkg.zip", 'r') as z:
        z.extractall("geodata")

### Using GeoPandas

GeoPandas is an open source project to make working with geospatial data in python easier. GeoPandas extends the datatypes used by pandas to allow spatial operations on geometric types. Geometric operations are performed by shapely. Geopandas further depends on fiona for file access and descartes and matplotlib for plotting.

In [ ]:
import geopandas as gpd

filename = "geodata/packages/natural_earth_vector.gpkg"
layer = "ne_10m_admin_0_countries"

df = gpd.read_file(filename, layer=layer)

In [ ]:
df.plot(figsize=(16,9));

In [ ]:
df.head(5)

In [ ]:
df.columns

In [ ]:
hk = df.query("ISO_A3 == 'HKG'")

In [ ]:
hk = df.query("NAME == 'Hong Kong'")

In [ ]:
hk.plot();

##### Retrieve Shapely Multipolygon

In [ ]:
hk_shape = hk.iloc[0]["geometry"]

In [ ]:
type(hk_shape)

In [ ]:
hk_shape

In [ ]:
shape.wkt

In [ ]:
from shapely.geometry import Point

p = Point(114.172239, 22.295719 ) # Kowloon Hotel

In [ ]:
p.within(hk_shape)

In [ ]:
import folium

In [ ]:
m = folium.Map(location=[22.295719, 114.172239], zoom_start=16)
m

In [ ]:
m = folium.Map(location=[22.295719, 114.172239], zoom_start=16)

folium.Marker([22.295719, 114.172239], popup="The Kowloon Hotel").add_to(m)
m

Adding our Shape

In [ ]:
m = folium.Map(location=[22.295719, 114.172239], zoom_start=10)

jsondata = hk.to_json()

folium.GeoJson(
    jsondata,
    style_function=lambda feature: {
        'fillColor': '#ffff00',
        'color': 'black',
        'weight': 2,
        'dashArray': '5, 5'
    }
).add_to(m)

m

### Retrieve Street Network from OSM Data using osmnx

* drive - get drivable public streets (but not service roads)
* drive_service - get drivable streets, including service roads
* walk - get all streets and paths that pedestrians can use (this network type ignores one-way directionality)
* bike - get all streets and paths that cyclists can use
* all - download all non-private OSM streets and paths
* all_private - download all OSM streets and paths, including private-access ones

In [ ]:
import osmnx as ox

In [ ]:
G = ox.graph_from_place('Hong Kong Island', which_result=2, network_type='drive')

In [ ]:
fig, ax = ox.plot_graph(G)

In [ ]:
nodes, streets = ox.graph_to_gdfs(G)

In [ ]:
len(streets)

In [ ]:
m = folium.Map([22.295719, 114.172239],
zoom_start=13,tiles="CartoDB dark_matter")

jsondata = streets.to_json()

style = {'color': '#FFDD66', 
         'weight':'1'}
folium.GeoJson(jsondata, style_function=lambda x: style).add_to(m)
m

### Find ATMs

see: https://wiki.openstreetmap.org/wiki/Key:amenity

In [ ]:
atm = ox.pois_from_place("Hong Kong Island", amenities=['atm'])

In [ ]:
len(atm)

In [ ]:
atm

In [ ]:
import html

m = folium.Map([22.295719, 114.172239],zoom_start=15)


def atm2poi(x):
    lng = x["geometry"].x
    lat = x["geometry"].y
    operator = x["operator"]

    if type(operator) == str:
        name = html.escape(operator)
    else:
        name = "unkown bank"
   
    folium.Marker([lat,lng], 
              popup=name,
              icon=folium.Icon(color="green", prefix="fa", icon="credit-card")).add_to(m)


atm.apply(atm2poi, axis=1)

m

### Other Data

In [ ]:
restaurant = ox.pois_from_place("Hong Kong Island", amenities=['restaurant'])

In [ ]:
len(restaurant)

In [ ]:
restaurant